Author: Huangrui Chu

Bachelor: Duke Kunshan University

Master: Yale University

Reference Book: 《The Essence Of Computing》Author: Jun Wu

Reference Blog: https://zhuanlan.zhihu.com/p/489842614 Author:鳄鱼

# 思考题2.3

Q1. 写出简单计算器的伪代码。一星/五星

Q2. 回旋打印二叉树的节点。（吴军使用过的面试题） 三星/五星

修改而常熟的官渡遍历算法，使得偶数行的节点从左向右遍历，奇数行的节点从右向左遍历。比如图2.8所示的二叉树，遍历的的顺序为1->2->3->6->5->4->7。

![图一](./图2.8典型的二叉树.png)

### 我的解题

### Q1

### 简单计算器问题 （在我的例题11.6 计算器问题有后续思考的代码）

实现一个简单计算器，它支持不带括号的四则运算，计算器要满足先乘除后加减的规则。

通常计算器的输入是一串字符，我们会从左到右扫描这个字符，扫到哪里就用一个指针指向哪里。

这里运用的堆栈的先进后出的规则，并且利用了python可以储存任意type的数据的优点。

使用了dictionary 来储存 各个运算符号的 优先级， 优先级越高，数值越大。

细节1：使用计算器时，数字是一个键一个键输入进去的。如果输入14.54，其实输入了5个字符，只不过它们被转换成一个浮点数而已。

吴军老师的话：“在设计计算器时，要留意如何将字符串转换成数字，要记住数字转换的优先级高于任何计算符即可。

细节2：书中的例题是5+4*3/4，因此在遇到"/"这个与"*"相同优先级的符号时，是会先计算"*"不错。但如果是 5+4*3-4= 呢？其实我们在遇到"-"时就应该去计算"*"而不是等待遇到下一个"*"或者"/"，在这个情况，要考虑到前面可能有积压在栈里的 5 + 这个待运算的部分，因此，在算完这一部分"*"时要记得检查 堆栈 是否清空， 不然，等遇到 5-12-4的时候就难处理了（如果按照之前的算法，答案就会是5-8=-3）

细节3：遇到"="时在计算完结果后，不要把结果压入栈后就结束，而是要去检查堆栈是否清空。以5+4*3/4=为例，遇到等号，会计算出 12/4=3，那么堆栈就剩[5,+,3] 不为空，不能把3 直接当成整个式子的计算结果直接输出。

新知识：判断deque是否为空

if deque:

    #不为空
    
else:

    #deque空了

In [12]:
from collections import deque #引入stack 堆栈

class SimpleOperator:
    def __init__(self,Operator_Precedence):
        self.OP= Operator_Precedence #Operator Precedence=OP, type: dictionary
        self.stack=deque()
        
    def update_Operator_Precedence(self,Operator_Precedence):
        self.OP=Operator_Precedence
    
    def __calculate(self):
        b=self.stack.pop()#because of the FILO, the b is the first number that pop out
        operator=self.stack.pop()# pop out the operator
        a=self.stack.pop()#because of the FILO, the a is the second number that pop out
        if operator=="+":
            self.stack.append(a+b)
        elif operator=="-":
            self.stack.append(a-b)
        elif operator=="*":
            self.stack.append(a*b)
        elif operator=="/":
            if b==0:
                raise ValueError("could not devided by 0")
            self.stack.append(a/b)
            
    def calculate(self,input_string):
        self.input_string=input_string
        previous_is_number=False
        is_float=False
        self.top_operator=None #储存栈内最顶部的运算符
        for i in range(len(self.input_string)):
            char=self.input_string[i]
            print(char)
            if char>="0" and char<="9":#如果是数字的话，就存入stack
                if previous_is_number:
                    if is_float:
                        previous_number=str(self.stack.pop())
                        current_number=float(previous_number+char)
                        self.stack.append(current_number)
                        print(self.stack)
                    else:
                        previous_number=str(self.stack.pop())
                        current_number=int(previous_number+char)
                        self.stack.append(current_number)
                        is_float=False
                        print(self.stack)
                else:
                    self.stack.append(int(char))
                    print(self.stack)
                    previous_is_number=True
                    is_float=False
            else:#如果是运算符的话，遇到同级的，就要开始前面的计算。
                #第一步，把前面的
                previous_is_number=False
                if char==".":
                    #最高优先级：数字转换 （例如 15.43）
                    previous_is_number=True #数字转换后，前面仍为数字
                    is_float=True
                    current_number=str(self.stack.pop())+"."
                    self.stack.append(current_number)
                    print(self.stack)
                elif self.top_operator==None:
                    self.top_operator=char
                    self.stack.append(char)
                    print(self.stack)
                elif char == "=":
                    self.__calculate()
                    result=self.stack.pop()
                    #check whether the stack is empty or not  
                    if self.stack:#if the stack is not empty, we need to do one more step of calculation
                        print(self.stack)
                        self.stack.append(result)
                        self.__calculate()
                        result=self.stack.pop()
                    #Then we can print out the cutcome
                    print("The result is {}".format(result))
                    print(self.stack)
                else:
                    if self.OP[char]==self.OP[self.top_operator]:
                        self.__calculate()
                        self.stack.append(char)
                        self.top_operator=char
                        print(self.stack)
                    elif self.OP[char]>self.OP[self.top_operator]:
                        self.top_operator=char
                        self.stack.append(char)
                        print(self.stack)
                    if self.OP[char]<self.OP[self.top_operator]:
                        #比如5-4*3+2= 遇到+号时
                        if self.top_operator!="(":
                            #只要不是（，我们就可以去计算
                            self.__calculate()
                            result=self.stack.pop()
                            print(result)
                            print(self.stack)
                            if self.stack:
                                #如果是因为有“*”，“/”积压下的计算
                                self.stack.append(result)
                                self.__calculate()
                            self.stack.append(char)
                            self.top_operator=char

In [13]:
Operator_Precedence=dict()
Operator_Precedence["+"]=1
Operator_Precedence["-"]=1
Operator_Precedence["*"]=2
Operator_Precedence["/"]=2

In [14]:
CHR_operator=SimpleOperator(Operator_Precedence)

In [15]:
input_string="5-4.0*3.57+45.2577="

In [16]:
CHR_operator.calculate(input_string)

5
deque([5])
-
deque([5, '-'])
4
deque([5, '-', 4])
.
deque([5, '-', '4.'])
0
deque([5, '-', 4.0])
*
deque([5, '-', 4.0, '*'])
3
deque([5, '-', 4.0, '*', 3])
.
deque([5, '-', 4.0, '*', '3.'])
5
deque([5, '-', 4.0, '*', 3.5])
7
deque([5, '-', 4.0, '*', 3.57])
+
14.28
deque([5, '-'])
4
deque([-9.28, '+', 4])
5
deque([-9.28, '+', 45])
.
deque([-9.28, '+', '45.'])
2
deque([-9.28, '+', 45.2])
5
deque([-9.28, '+', 45.25])
7
deque([-9.28, '+', 45.257])
7
deque([-9.28, '+', 45.2577])
=
The result is 35.9777
deque([])


In [2]:
class Node:
    def __init__(self,value):
        self.value=value
        self.left=None
        self.right=None
        
import queue
def BreathFirstTraverseTree(node):
    if node==None:
        return
    temp_queue=queue.Queue()
    temp_queue.put(node)
    while(not temp_queue.empty()):
        node=temp_queue.get()
        print(node.value)
        if node.left!=None:
            temp_queue.put(node.left)
        if node.right!=None:
            temp_queue.put(node.right)

In [3]:
root=Node(1)
root.left=Node(2)
root.right=Node(3)
root.left.left=Node(4)
root.left.right=Node(5)
root.right.left=Node(6)
root.right.left.right=Node(7)

In [4]:
BreathFirstTraverseTree(root)

1
2
3
4
5
6
7


In [23]:
class stack_queue():
    def __init__(self):
        self.list=[]
    def pop(self):
        return self.list.pop()
    def get(self):
        result=self.list[0]
        self.list=self.list[1:]
        return result
    def append(self,node):
        self.list.append(node)
    def insert(self,node):
        self.list=[node]+self.list

#### 解题思路

stack queue 交替使用。 为了方便设计了stack_queue这样的储存结构。

为了标注每一行的node，使用了queue去储存level

In [24]:
import queue
def CyclePrint(node):
    if node==None:
        return
    nodes=stack_queue()
    nodes.append(node)
    level_queue=queue.Queue()
    level_queue.put(1)
    while(not level_queue.empty()):
        level=level_queue.get()
        if level%2==1:
            node=nodes.get()
            print(node.value)
            #父节点为基数，则按照右左添加子叶
            if node.right!=None:
                nodes.append(node.right)
                level_queue.put(level+1)
            if node.left!=None:
                nodes.append(node.left)
                level_queue.put(level+1)
        else:
            node=nodes.pop()
            print(node.value)
            #父节点为偶数，则按照左右添加子叶
            if node.left!=None:
                nodes.insert(node.left)
                level_queue.put(level+1)
            if node.right!=None:
                nodes.insert(node.right)
                level_queue.put(level+1)

       

In [25]:
CyclePrint(root)

1
2
3
6
5
4
7


#### 解题思路2

stack queue 交替使用。 发现collections.deque 可以实现popleft() 而且是O（1）如果到时候可以import 也可以试试


为了标注每一行的node，使用了queue去储存level